# Twitter Sentiment Analysis

#### Goal: To create a model that can effectively predict sentiment (Positive or Negative) in tweets.

Data is from Sentiment140 which provides 1.6 million labeled Tweets.

### Plan
1. Take a small subset of the data in order to tune the XGB model
2. Once the best parameters are chosen, will use grid search to tune the vectorizer.
3. Increase the amount of data to train the model on.
4. Perform Word2Vec on the data.

In [1]:
############################### Imports ##################################

# Basic
import numpy as np
import pandas as pd
import scipy
import re
import time
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import boto3
import io
import warnings
warnings.filterwarnings('ignore')

# NLP
import nltk
import spacy
spacy.load('en')
from nltk.corpus import stopwords
import preprocessor as p

# Model Infrastructure
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report


# Models
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [2]:
#################################### Bring in Data #############################################
start_time = time.time()
s3 = boto3.client('s3')

#Bring in Training Data
obj = s3.get_object(Bucket='data-science-project-data', Key='Twitter_Sentiment_Analysis/training.1600000.processed.noemoticon.csv')
cols = ['sentiment','id','date','query_string','user','text']
tweets = pd.read_csv(io.BytesIO(obj['Body'].read()),header=None, names=cols, encoding = "ISO-8859-1")
#train.set_index('bidder_id', inplace=True)

print("-- Execution time: %s seconds ---" % (time.time() - start_time))

-- Execution time: 6.573343515396118 seconds ---


In [3]:
tweets.head()

,sentiment,id,date,query_string,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
# Just Need the Sentiment and the Text
tweets.drop(['id','date','query_string','user'],axis=1,inplace=True)

In [5]:
# Clean the tweets
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.EMOJI)
def preprocess_tweet(tweet):
    return p.clean(tweet)

# Clean the tweets, by removing special characters
start_time = time.time()
tweets['Clean'] = tweets['text'].apply(lambda x: preprocess_tweet(x))
print("-- Execution time: %s seconds ---" % (time.time() - start_time))

-- Execution time: 60.021862268447876 seconds ---


In [6]:
# Down Sample
tweets_subsampled_1, tweets_subsampled_2 = train_test_split(tweets, test_size=0.1)

In [7]:
#Split between outcome and Features
y = tweets_subsampled_2['sentiment']
X = tweets_subsampled_2['Clean']

### Model Training
Will use tuning to find the optimal parameters for the model.

#### Transform Data

In [8]:
start_time = time.time()
# Create lemmatizer using spacy
lemmatizer = spacy.lang.en.English()

def custom_tokenizer(doc):
    tokens = lemmatizer(doc)
    return([token.lemma_ for token in tokens if not token.is_punct])

pipe = Pipeline(steps=[('vectidf', TfidfVectorizer(tokenizer=custom_tokenizer, stop_words='english',
                                                   lowercase=True,use_idf=True,max_df=0.5,
                                                  min_df=2, norm='l2', smooth_idf=True)),
                 ('svd', TruncatedSVD(500)),
                 ('norm',Normalizer(copy=False))
                       ])

tweets_transform = pipe.fit_transform(X)

print("-- Execution time: %s seconds ---" % (time.time() - start_time))

-- Execution time: 61.48358964920044 seconds ---


In [9]:
#splitting into training and test sets
X_train, X_test, y_train, y_test = train_test_split(tweets_transform,y,test_size=0.25)

#### Grid Search Model

In [13]:
####### Base Line Model ########
warnings.filterwarnings('ignore')
start_time = time.time()
xgb_model = XGBClassifier(random_state=10) 
 

parameters = {'n_jobs':[20],
             }

clf = GridSearchCV(xgb_model, parameters,cv=3, verbose=0,n_jobs=1)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
print("-- Execution time: %s seconds ---" % (time.time() - start_time))

0.693325
-- Execution time: 70.32373976707458 seconds ---


In [14]:
# Tune n_estimators given the learning rate
# n_estimators is the number of trees to use
# learning_rate is to make the model more robust by shrinking the weights on each step. It determines the impact of each tree on the final outcome
warnings.filterwarnings('ignore')
start_time = time.time()
xgb_model = XGBClassifier(learning_rate =0.1,
                             max_depth=5,
                             min_child_weight=1,
                             gamma=0,
                             subsample=0.8,
                             colsample_bytree=0.8,
                             scale_pos_weight=1,
                             random_state=10) 
 

parameters = {'n_jobs':[30],
             'n_estimators':range(100,1000,200)
             }

clf = GridSearchCV(xgb_model, parameters,cv=3, verbose=0,n_jobs=1)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
print("-- Execution time: %s seconds ---" % (time.time() - start_time))

0.72365
-- Execution time: 1408.1651225090027 seconds ---


In [15]:
# Find Best Parameters
clf.best_params_

{'n_estimators': 500, 'n_jobs': 30}

In [16]:
# Tune Max_depth and min_child_weight
# Max_depth is the maximum depth of the tree. Note the more, the more likelhood for overfitting
# min_child_weight Defines the minimum sum of weights of all observations required in a child. its used to control overfitting as well
warnings.filterwarnings('ignore')
start_time = time.time()
xgb_model = XGBClassifier(learning_rate =0.1,
                             n_estimators=500,
                             gamma=0,
                             subsample=0.8,
                             colsample_bytree=0.8,
                             scale_pos_weight=1,
                             random_state=10) 
 

parameters = {'n_jobs':[35],
             'max_depth':range(3,10,2),
             'min_child_weight':range(1,6,2)
             }

clf = GridSearchCV(xgb_model, parameters,cv=3, verbose=0,n_jobs=1)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
print("-- Execution time: %s seconds ---" % (time.time() - start_time))

0.72605
-- Execution time: 3803.7127678394318 seconds ---


In [17]:
# Find Best Parameters
clf.best_params_

{'max_depth': 5, 'min_child_weight': 5, 'n_jobs': 35}

In [18]:
# Tuned Gamma
# Gamma specifies the minimum loss reduction required to make a split (on positive reduction in the loss function).
warnings.filterwarnings('ignore')
start_time = time.time()
xgb_model = XGBClassifier(learning_rate =0.1,
                             n_estimators=500,
                             max_depth=5,
                             min_child_weight=5,
                             subsample=0.8,
                             colsample_bytree=0.8,
                             scale_pos_weight=1,
                             random_state=10) 
 

parameters = {'n_jobs':[35],
             'gamma':[i/10.0 for i in range(0,5)]
             }

clf = GridSearchCV(xgb_model, parameters,cv=3, verbose=0,n_jobs=1)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
print("-- Execution time: %s seconds ---" % (time.time() - start_time))


0.727625
-- Execution time: 1406.7063238620758 seconds ---


In [19]:
# Find Best Parameters
clf.best_params_

{'gamma': 0.1, 'n_jobs': 35}

In [22]:
# Tune subsample and colsample_bytree
# Subsample denotes the fraction of observates to be randomly sampled. Lower values make algorithm more conservative 2 small leads to underfitting
# colsample_bytree denotes the fraction of coumns to be randomly sampled for each tree

warnings.filterwarnings('ignore')
start_time = time.time()
xgb_model = XGBClassifier(learning_rate =0.1,
                             n_estimators=500,
                             max_depth=5,
                             min_child_weight=5,
                             gamma=0.1,
                             scale_pos_weight=1,
                             random_state=10) 
 

parameters = {'n_jobs':[200],
             'subsample':[i/100.0 for i in range(75,90,5)],
             'colsample_bytree':[i/100.0 for i in range(75,90,5)]
             }

clf = GridSearchCV(xgb_model, parameters,cv=3, verbose=0,n_jobs=1)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
print("-- Execution time: %s seconds ---" % (time.time() - start_time))

0.727625
-- Execution time: 5729.693859577179 seconds ---


In [23]:
# Find Best Parameters
clf.best_params_

{'colsample_bytree': 0.8, 'n_jobs': 200, 'subsample': 0.8}

In [24]:
# Tune the regularization parameter
# reg_alpha is the L1 regularization on the weights. Can be used with high dimensionality so it runs faster.
# reg_lambda is the L2 regularization on the weights

warnings.filterwarnings('ignore')
start_time = time.time()
xgb_model = XGBClassifier(learning_rate =0.1,
                             n_estimators=500,
                             max_depth=5,
                             min_child_weight=5,
                             gamma=0.1,
                             subsample=0.8,
                             colsample_bytree=0.8, 
                             scale_pos_weight=1,
                             random_state=10) 
 

parameters = {'n_jobs':[200],
             'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05],
             'reg_lambda':[0, 0.1, 0.5, 1]
             }

clf = GridSearchCV(xgb_model, parameters,cv=3, verbose=0,n_jobs=1)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
print("-- Execution time: %s seconds ---" % (time.time() - start_time))


KeyboardInterrupt: 

In [ ]:
# Find Best Parameters
clf.best_params_

In [ ]:
# Tune estimators and the learning parameter
warnings.filterwarnings('ignore')
start_time = time.time()
xgb_model = XGBClassifier(   max_depth=5,
                             min_child_weight=5,
                             gamma=0.1,
                             subsample=0.8,
                             colsample_bytree=0.8, 
                             scale_pos_weight=1,
                             random_state=10) 
 

parameters = {'n_jobs':[200],
             'n_estimators':range(500,5001,500),
             'learning_rate':[0, 0.0001, 0.001, 0.01, 0.1, 0.2]
             }

clf = GridSearchCV(xgb_model, parameters,cv=3, verbose=0,n_jobs=1)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
print("-- Execution time: %s seconds ---" % (time.time() - start_time))

### Vectorizer Tuning

In [ ]:
#splitting into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)

In [ ]:
start_time = time.time()
# Create lemmatizer using spacy
lemmatizer = spacy.lang.en.English()

def custom_tokenizer(doc):
    tokens = lemmatizer(doc)
    return([token.lemma_ for token in tokens if not token.is_punct])

xgb_model = XGBClassifier(   max_depth=5,
                             min_child_weight=5,
                             gamma=0.1,
                             subsample=0.8,
                             colsample_bytree=0.8, 
                             scale_pos_weight=1,
                             random_state=10,
                             n_jobs=200) 

pipe = Pipeline(steps=[('vectidf', TfidfVectorizer(tokenizer=custom_tokenizer, stop_words='english',lowercase=True,use_idf=True,max_df=0.5)),
                 ('svd', TruncatedSVD(500)),
                 ('norm',Normalizer(copy=False)),
                 ('xgb',xgb_model)
                 ])

parameters = {'vectidf__ngram_range': [(1, 1), (1, 2)],
              'vectidf__min_df':(1,2),
              'vectidf__norm':['l1','l2'],
              'vectidf__smooth_idf':[True, False]
              }

grid = GridSearchCV(pipe, parameters, n_jobs=1, cv=3, verbose=0)
grid.fit(X_train, y_train)

print("-- Execution time: %s seconds ---" % (time.time() - start_time))

### Increasing Training Data

In [ ]:
# Down Sample
tweets_subsampled_1, tweets_subsampled_2 = train_test_split(tweets, test_size=0.3)

#Split between outcome and Features
y = tweets_subsampled_2['sentiment']
X = tweets_subsampled_2['Clean']

#splitting into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)

In [ ]:
# Train Model
start_time = time.time()
# Create lemmatizer using spacy
lemmatizer = spacy.lang.en.English()

def custom_tokenizer(doc):
    tokens = lemmatizer(doc)
    return([token.lemma_ for token in tokens if not token.is_punct])

xgb_model = XGBClassifier(   max_depth=5,
                             min_child_weight=5,
                             gamma=0.1,
                             subsample=0.8,
                             colsample_bytree=0.8, 
                             scale_pos_weight=1,
                             random_state=10,
                             n_jobs=200) 

pipe = Pipeline(steps=[('vectidf', TfidfVectorizer(tokenizer=custom_tokenizer, stop_words='english',lowercase=True,use_idf=True,max_df=0.5)),
                 ('svd', TruncatedSVD(2000)),
                 ('norm',Normalizer(copy=False)),
                 ('xgb',xgb_model)
                 ])

parameters = {'vectidf__ngram_range': [(1, 1), (1, 2)],
              'vectidf__min_df':(1,2),
              'vectidf__norm':['l1','l2'],
              'vectidf__smooth_idf':[True, False]
              }

grid = GridSearchCV(pipe, parameters, n_jobs=1, cv=3, verbose=0)
grid.fit(X_train, y_train)

print("-- Execution time: %s seconds ---" % (time.time() - start_time))